We first load the packages that we will use in this notebook

In [ ]:
from pathlib import Path # Easy path manipulation
import pandas as pd      # Data handling
import igraph as ig      # Network analysis
import leidenalg as la   # Clustering

# Creating the network

We will be working with the same set of publications. These are all publications on scientometrics/bibliometrics. In addition to the publications themselves, all citation relations between these publications are now also available. As previously, we will read in the data using the `pandas` library (made available as `pd`).

In [ ]:
data_dir = Path('.') / '..' / 'data'

# Read publications
publications_df = pd.read_csv(data_dir / 'publications.txt', sep='\t', encoding='utf-8')

<div class="alert alert-info">
    Read the data in <code>citations.txt</code> and store it in the variable <code>citations_df</code>.
</div>

Let's have a look again at what information was contained in `publications_df`.

In [ ]:
publications_df.head(5)

<div class="alert alert-info">
    Checkout what is in <code>citations_df</code>.
</div>

We now have all the information we need to create a citation network. Each node will be a publication, and each edge will be a citation. The citations will be directed, pointing from the citing publication to the cited publication.

We will construct the network using `ig.Graph.DataFrame`. This method will consider the first column of `publication_df` as the name of the node, and the first two columns of `citations_df` as respectively the source and target name of each node, corresponding to the citing and the cited publication in our case.

In [ ]:
G = ig.Graph.DataFrame(edges=citations_df, directed=True,
                       vertices=publications_df)

It appears we have some missing publications in `publications_df`. That is, some edges refer to a `paper_id` that is not contained in `publications_df`. The reason is simple: `citations_df` contains all references from all papers in `publications_df`, that is, also including references to publications that are not contained in `publications_df`. Let us get the the citations that are only within the set of papers in `publications_df`.

In [ ]:
internal_citations_df = citations_df[citations_df['cited_paper_id'].isin(publications_df['paper_id'])]
G = ig.Graph.DataFrame(edges=internal_citations_df, directed=True,
                       vertices=publications_df)

Now `G` contains our citation network, and you can start doing some analysis. Let us first check if the number of nodes and edges are as we expect them to be based on the dataframes that we loaded. In `igraph` nodes are called *vertices* and links are called *edges*.

In [ ]:
# Check number of nodes
print("Number of nodes: ", G.vcount())
print("Number of rows in publications_df: ", publications_df.shape[0])

# Check number of edges
print("Number of edges: ", G.ecount())
print("Number of rows in citations_df: ", citations_df.shape[0])

# Basic graph operations

As said, `ig.Graph.DataFrame` considers the first column of `publications_df` to be the vertex name. Each node is also identified with a specific integer position, as usual 0-based. We can query the first node as follows:

In [ ]:
G.vs[0]

This now shows a lot of information, most of it coming from the `publications_df`, and they are stored as vertex attributes. You can also just print the attributes, which provides a bit better overview.

In [ ]:
G.vs[0].attributes()

As you can see the `name` of the vertex is `1481577014`, which was the first row in `publications_df`. Also all the other information that was contained in `publications_df` is now also available as vertex attributes. Something similar holds for edges, but in this case we didn't supply any additional attributes, such as a weight.

In [ ]:
G.es[0]

The endpoints of the edge are available as `source` and `target` respectively:

In [ ]:
G.es[0].source, G.es[0].target

These are just integer identifiers of the nodes, and you can see to which publication they refer by passing it back to the `vs`:

In [ ]:
G.vs[G.es[0].source]

You can easily also get the information of vertex attributes for all vertices. For example, if we want to get the `name` attribute for all vertices we can do:

In [ ]:
G.vs['name']

This is now just an ordinary Python list.

Sometimes it is necessary to selection a certain subset of nodes, and restrict the graph to those nodes. For instance, perhaps we are interested only in publications that have appeared in `Scientometrics`:

In [ ]:
scientometrics_nodes = G.vs.select(journal_title_eq = 'Scientometrics')

Here, in `G.vs.select` the `journal_title` refers to a vertex attribute, while `eq` indicates that we want to select only nodes for which `journal_title` is *eq*ual to the provided value `'Scientometrics'`. As usual, you can always check out the documentation for more information (just hit `Shift-Tab` when the cursor is on the function).

<div class="alert alert-info">
    Try to list all <code>paper_title</code>s in <code>scientometrics_nodes</code>.
</div>

We can now easily get the (induced) subgraph of only these nodes. The subgraph contains all the indicated nodes and all links between these nodes.

In [ ]:
G_scientometrics = G.subgraph(scientometrics_nodes)
print("Nodes:", G_scientometrics.vcount(), "Edges:", G_scientometrics.ecount())

Let us start doing more structural analyses. Let us see if two nodes are connected by some (shortest) citation path.

In [ ]:
paths = G.get_shortest_paths(0, 1)
paths

The results show that there is one shortest path between node 0 and node 1. The entries of this list contain the node identifiers of all the publications encountered on this shortest path.

In [ ]:
G.vs[paths[0]]['paper_title']

<div class="alert alert-info">
    Get the path between node <code>0</code> and node <code>2</code>.
</div>

You now get a warning, saying that some nodes cannot be reached. This means that not all publications are connected to each. If there are paths between all pairs of nodes, the graph is said to be connected. In our case, our graph is apparently not connected.

In [ ]:
G.is_connected()

Let us consider the separate parts of the network that are connected. These separate parts are called connected components (rather confusingly called `clusters` in `igraph`). In this case, we ignore the direction for all links, and just wonder whether nodes are connected through an undirected path. These are called "weakly" connected components.

In [ ]:
components = G.clusters(mode=ig.WEAK)

Now `components` contains a partition of the nodes into separate connected components. Let us see how many components there are

In [ ]:
len(components)

That is quite a bit of different number of components. Let's check out how large they all are

In [ ]:
components.sizes()

As you can see, there is one very large component, consisting of 29219 nodes, while most others consist of only a single publication. Let us take the subgraph of the largest connected components, often called the *giant connected component*.

In [ ]:
H = components.giant()

<div class="alert alert-info">
    How many nodes and edges are in the giant component <code>H</code>?
</div>

Now let us check if `H` is connected, as expected

In [ ]:
H.is_connected()

<div class="alert alert-info">
    Any idea why <code>H.is_connected()</code> returned <code>False</code>? Can you correct the code?
</div>

# Clustering

The Leiden algorithm is implemented directly in `igraph`. However, this is limited to undirected graphs only. The implementation available from `leidenalg` is more extensive, but is also a bit slower. We will first briefly work with the `igraph` implementation, and then switch to the `leidenalg` implementation.

We first need to make the graph undirected.

In [ ]:
H_u = H.copy()       # We first make a copy
H_u.to_undirected()  # because the graph is changed in-place

Before we cluster the network, we want to ensure that we have reproducible results. Each run of a clustering algorithm may return (slightly) different results. Especially to ensure we are all looking at identical results, we can therefore seed the random number generator.

In [ ]:
import random
ig.set_random_number_generator(random)

We can then simply cluster the network

In [ ]:
random.seed(0)
clusters = H_u.community_leiden()

The variable `clusters` is of the same type as the `components` that we saw earlier. Both provide a partition of a network.

<div class="alert alert-info">
    How many clusters were found?
</div>

The number of clusters is exactly equal to the number of nodes! This is because the method `community_leiden` by default uses a specific method (CPM) for which you explicitly need to set a specific resolution parameter. The deafult resolution parameter is 1, and it then typically results in equally many clusters as nodes. Let us try again with some smaller resolution parameters

In [ ]:
random.seed(0)
clusters = H_u.community_leiden(resolution_parameter=0.1)
len(clusters)

This is perhaps still a bit much. Let's reduce it further.

In [ ]:
random.seed(0)
clusters = H_u.community_leiden(resolution_parameter=0.01)
len(clusters)

This is still a bit much, but perhaps there are simply a lot of small clusters, similar to what we saw in the `components`.

In [ ]:
sorted(clusters.sizes(), reverse=True)

This is perhaps a reasonable resolution to explore a bit further. Now we are just going to get the indices of each cluster in sort order.

In [ ]:
import numpy as np
sizes = np.array(clusters.sizes())
size_rank = np.argsort(sizes)[::-1]
size_rank[:10]

Apparently cluster 227 is the largest, then cluster number 127, folowed by 267 et cetera. Let's check the sizes of those 10 largest clusters to confirm.

In [ ]:
sizes[size_rank[:10]]

Now we can get separate subgraphs for all clusters. Let us first take a closer loook to the largest cluster

In [ ]:
H_cluster = clusters.subgraph(size_rank[0])
print("Nodes:", H_cluster.vcount(), "Edges:", H_cluster.ecount())

This is also sufficiently small to be able to plot the network.

In [ ]:
ig.plot(H_cluster,
        vertex_size=7, 
        vertex_frame_color='white', edge_color='light gray')

As you can see, this is quite a hairball. This will be the case for most of the clusters: they are fairly well connected among each other after all.

Let us try to make sense of this cluster. Let us take a look at some publications in the cluster

In [ ]:
cluster_pubs = H_cluster.get_vertex_dataframe()
cluster_pubs.sort_values('n_cits', ascending=False)[['authors', 'journal_title', 'pub_year', 'paper_title']].head(20)

This cluster seems to revolve around the h-index.

<div class="alert alert-info">
    See for yourself what clusters 1-4 are about.
</div>

Let us see how we can work with `leidenalg`. This package is a bit more flexible, and we will see during the VOSviewer practical session how this will prove useful. We now no longer have to work with the undirected network `H_u`, since the `leidenalg` package also supported directed edges. In this case the directionality doesn't matter for the results though.

In [ ]:
clusters = la.find_partition(H, la.CPMVertexPartition, resolution_parameter=0.01, seed=0)
len(clusters)

The second argument to `find_partition`, specifies the type of objective function we try to optimise, which in this case is the `CPMVertexPartition`. There are also other objective functions that `leidenalg` can optimise, but we will not cover those.

Let us check again the sizes

In [ ]:
clusters.sizes()

As you can see, these results are already sorted, so there's not need to sort them ourselves. The results are a bit different than from the `igraph` implementation, but let's see how different they are.

Let us again take a closer look to the largest cluster.

In [ ]:
H_cluster = clusters.subgraph(0)
cluster_pubs = H_cluster.get_vertex_dataframe()
cluster_pubs.sort_values('n_cits', ascending=False)[['authors', 'journal_title', 'pub_year', 'paper_title']].head(20)

<div class="alert alert-info">
    Again see for yourself what clusters 1-4 are about. What are the differences with the results obtained using <code>igraph</code>?
</div>

## Weighted clustering

For clustering publications throughout an entire bibliographic database we normally use edge weights of $\frac{1}{r_i}$ where $r_i$ is the number of references of publication $i$. Let us also create those weights for this network.

First, we need to get the number of references. We already have the number of references available in the vertex attribute `n_refs`. This is the total number of references, also including many citation links that are not included in this network. The number of links of a node is called the *degree*. In this case, we are specifically interested in the number of outgoing links, called the *out degree*. Let us compare this out degree to the total number of references `n_refs`.

In [ ]:
out_degree = np.array(H.vs.degree(mode='out'))
in_degree = np.array(H.vs.degree(mode='in'))
n_refs = np.array(H.vs['n_refs'])

Let's see what proportion of the references are included in this dataset.

In [ ]:
int_cov = out_degree/n_refs
np.mean(int_cov[~np.isnan(int_cov)])
H.vs['int_cov'] = int_cov

In [ ]:
H.es['weight'] = [1.0/out_degree[e.source] for e in H.es]

The sum of all weights, is called the *weighted degree* or *strength*. If we did our calculations correctly, the total weight of all outgoing links per node should now sum to 1. Let's check.

In [ ]:
H.strength(mode='out', weights='weight')

Now let us cluster the network using these weights.

In [ ]:
clusters = la.find_partition(H, la.CPMVertexPartition, resolution_parameter=0.01, weights='weight', seed=0)
len(clusters)

We now get many more clusters. This is because we are now using weights that are much lower than 1. If no weights are specified, a default weight of 1 is implicitly used. Let us therefore decrease the resolution parameter further again.

In [ ]:
clusters = la.find_partition(H, la.CPMVertexPartition, resolution_parameter=0.0005, weights='weight', seed=0)
len(clusters)

In [ ]:
clusters.sizes()

<div class="alert alert-info">
    Check out the clusters again. Do you think the difference are small or large, compared to the previous results without weights? Checkout cluster number 4. What seems special about it?
</div>

## Extended direct citation

So far, we have only concentrated on the citations within the set of publications given by `publications_df`. However, the external citations, referring to publications *not* contained in `publications_df`, may also contain relevant information. For example, two publications from `publications_df` may not appear strongly related when looking at internal citations only, while they are connected through many external citations. This approach is sometimes called "extended direct citation". This is also what we use when clustering publications in an entire bibliometric database.

Let us first establish the necessary dataframe that contains all relevant publications (both internal and external).

In [ ]:
# Get unique paper_id of both citing and cited paper by stacking them
all_pubs_df = citations_df.stack().reset_index(drop=True).drop_duplicates()
all_pubs_df.name = 'paper_id'

# Merge it with original publication information
all_pubs_df = pd.merge(all_pubs_df, publications_df, on='paper_id', how='outer', indicator='type')

# Clarify type of publication: a publication is external if it isn't included in 
# the original publications_df, but it only included because it is a reference of
# one of those publications
all_pubs_df['type'] = all_pubs_df['type'].replace({'both': 'internal',
                                                   'right_only': 'internal',
                                                   'left_only': 'external'})

We can now create the extended graph, using all citations contained in `citations_df`.

In [ ]:
G_extended = ig.Graph.DataFrame(edges=citations_df, directed=True,
                                vertices=all_pubs_df)
print("Nodes:", G_extended.vcount(), "Edges:", G_extended.ecount())

As you can see, there are now many more nodes in the extended graph. We previously had 30021 nodes, and we now have a little over 130 000 additional publications included.

The idea behind clustering using extended direct citations is that we are interested only in the internal publications, but we use the external publications to inform the clustering of the internal publications. Normally, the clustering method considers it a cost to create large clusters. This cost is essentially indicated by the `resolution_parameter`, and with a higher `resolution_parameter`, there is a larger cost, leading to smaller clusters. However, when using extended direct citations, we only care about this for the internal publications, not for the external publications. We therefore specify a weight of 1 for internal publications, and a weight of 0 for external publications. This means that the clustering will not take the inclusion of external publications as a cost.

In [ ]:
G_extended.vs['weight'] = [1 if v['type'] == 'internal' else 0 for v in G_extended.vs]

Let us again take the weighted approach we introduced earlier.

In [ ]:
out_degree = G_extended.degree(mode='out')
G_extended.es['weight'] = [1.0/out_degree[e.source] for e in G_extended.es]

We are now ready to run the clustering again. We now specifically use the `node_sizes` argument to provide the publication weights.

In [ ]:
clusters = la.find_partition(G_extended, la.CPMVertexPartition,
                             weights=G_extended.es['weight'], node_sizes=G_extended.vs['weight'],
                             resolution_parameter=0.0005, seed=0)

Let us again check out the largest cluster

In [ ]:
H_cluster = clusters.subgraph(11)
print("Nodes:", H_cluster.vcount(), " Node size: ", sum(H_cluster.vs['weight']), "Edges:", H_cluster.ecount())
cluster_pubs = H_cluster.get_vertex_dataframe()
cluster_pubs.sort_values('n_cits', ascending=False)[['authors', 'journal_title', 'pub_year', 'paper_title']].head(20)

There are in total 872 nodes in this cluster, but only 416 internal publications.

Let us plot this cluster for illustration.

In [ ]:
layout = H_cluster.layout_auto()
ig.plot(H_cluster, layout=layout,
        vertex_size=7, vertex_frame_color='white', edge_arrow_size=0.5, edge_color='light gray',
        vertex_color=H_cluster.vs['weight'], palette=ig.RainbowPalette(2))

The blue nodes are the internal publications, while the red nodes are the external publications. Many of the external publications are peripheral, but sometimes do seem to pull in some internal publications, which otherwise are only weakly related.